# Gold standard curation: Preprocessing and single-step regression

In this stage of gold standard curation, we will do the data preprocessing, selection, and single-step regression for the 153 traits in our question set. This file shows the reference steps using the trait "Breast Cancer" as an example. The workflow consists of the following steps:

1. Preprocess all the cohorts related to this trait. Each cohort should be converted to a tabular form and saved to a csv file, with columns being genetic factors, the trait, and age, gender if available;
2. If there exists at least one cohort with age or gender information, conduct regression analysis with genetic features together with age or gender as the regressors.


# 1. Basic setup

In [1]:
import os

# Set your preferred name
USER = "Jinglin"
# Set the data and output directories
DATA_ROOT = '../../DATA'
OUTPUT_ROOT = '../../output1'
TRAIT = 'Thyroid Cancer'

OUTPUT_DIR = os.path.join(OUTPUT_ROOT, USER, '-'.join(TRAIT.split()))
JSON_PATH = os.path.join(OUTPUT_DIR, "cohort_info.json")
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR, exist_ok=True)

# Gene symbol normalization may take 1-2 minutes. You may set it to False for debugging.
NORMALIZE_GENE = True

In [2]:
# This cell is only for use on Google Colab. Skip it if you run your code in other environments

"""import os
from google.colab import drive

drive.mount('/content/drive', force_remount=True)
proj_dir = '/content/drive/MyDrive/AI4Science_Public'
os.chdir(proj_dir)"""

"import os\nfrom google.colab import drive\n\ndrive.mount('/content/drive', force_remount=True)\nproj_dir = '/content/drive/MyDrive/AI4Science_Public'\nos.chdir(proj_dir)"

# 2. Data preprocessing and selection

## 2.1. The TCGA Xena dataset

In TCGA Xena, there is either zero or one cohort related to the trait. We search the names of subdirectories to see if any matches the trait. If a match is found, we directly obtain the file paths.

In [3]:
import os
from utils import *

dataset = 'TCGA'
dataset_dir = os.path.join(DATA_ROOT, dataset)
os.listdir(dataset_dir)[:10]

['TCGA_Pancreatic_Cancer_(PAAD)',
 'TCGA_Bladder_Cancer_(BLCA)',
 '.DS_Store',
 'TCGA_Testicular_Cancer_(TGCT)',
 'TCGA_Acute_Myeloid_Leukemia_(LAML)',
 'TCGA_Breast_Cancer_(BRCA)',
 'TCGA_Stomach_Cancer_(STAD)',
 'TCGA_Thyroid_Cancer_(THCA)',
 'TCGA_Glioblastoma_(GBM)']

If no match is found, jump directly to GEO in Part 2.2

In [4]:
trait_subdir = "TCGA_Thyroid_Cancer_(THCA)"
cohort = 'Xena'
# All the cancer traits in Xena are binary
trait_type = 'binary'
# Once a relevant cohort is found in Xena, we can generally assume the gene and clinical data are available
is_available = True

clinical_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_THCA_clinicalMatrix')
genetic_data_file = os.path.join(dataset_dir, trait_subdir, 'TCGA.THCA.sampleMap_HiSeqV2_PANCAN.gz')

In [5]:
import pandas as pd

clinical_data = pd.read_csv(clinical_data_file, sep='\t', index_col=0)
genetic_data = pd.read_csv(genetic_data_file, compression='gzip', sep='\t', index_col=0)
age_col = gender_col = None

In [6]:
_, clinical_data_cols = check_rows_and_columns(clinical_data)

In [7]:
clinical_data_cols[:10]

['_INTEGRATION',
 '_PATIENT',
 '_cohort',
 '_primary_disease',
 '_primary_site',
 'additional_pharmaceutical_therapy',
 'additional_radiation_therapy',
 'additional_surgery_locoregional_procedure',
 'age_at_initial_pathologic_diagnosis',
 'bcr_followup_barcode']

Read all the column names in the clinical dataset, to find the columns that record information about age or gender.
Reference prompt:

In [8]:
f'''
Below is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:
candidate_age_cols = [col_name1, col_name2, ...]
candidate_gender_cols = [col_name1, col_name2, ...]
If no columns match a criterion, please provide an empty list.

Column names:
{clinical_data_cols}
'''

"\nBelow is a list of column names from a biomedical dataset. Please examine it and identify the columns that are likely to contain information about patients' age. Additionally, please do the same for columns that may hold data on patients' gender. Please provide your answer by strictly following this format, without redundant words:\ncandidate_age_cols = [col_name1, col_name2, ...]\ncandidate_gender_cols = [col_name1, col_name2, ...]\nIf no columns match a criterion, please provide an empty list.\n\nColumn names:\n['_INTEGRATION', '_PATIENT', '_cohort', '_primary_disease', '_primary_site', 'additional_pharmaceutical_therapy', 'additional_radiation_therapy', 'additional_surgery_locoregional_procedure', 'age_at_initial_pathologic_diagnosis', 'bcr_followup_barcode', 'bcr_patient_barcode', 'bcr_sample_barcode', 'braf_gene_genotyping_outcome_lab_results_text', 'days_to_additional_surgery_metastatic_procedure', 'days_to_birth', 'days_to_collection', 'days_to_death', 'days_to_initial_pathol

In [9]:
candidate_age_cols = ['age_at_initial_pathologic_diagnosis',
                      'days_to_birth']
candidate_gender_cols = ['gender']

Choose a single column from the candidate columns that record age and gender information respectively.
If no column meets the requirement, keep 'age_col' or 'gender_col' to None

In [10]:
preview_df(clinical_data[candidate_age_cols])

{'age_at_initial_pathologic_diagnosis': [74, 65, 55, 57, 33],
 'days_to_birth': [-27348, -24102, -20361, -21008, -12295]}

In [11]:
age_col = 'age_at_initial_pathologic_diagnosis'

In [12]:
preview_df(clinical_data[candidate_gender_cols])

{'gender': ['FEMALE', 'MALE', 'MALE', 'MALE', 'FEMALE']}

In [13]:
gender_col = 'gender'

In [14]:
selected_clinical_data = xena_select_clinical_features(clinical_data, TRAIT, age_col, gender_col)

In [15]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

15 input query terms found dup hits:	[('GTF2IP1', 2), ('RBMY1A3P', 3), ('HERC2P2', 3), ('WASH3P', 3), ('NUDT9P1', 2), ('SNORD104', 2), ('
154 input query terms found no hit:	['C16orf13', 'C16orf11', 'LOC100272146', 'LOC339240', 'NACAP1', 'LOC441204', 'KLRA1', 'FAM183A', 'FA
9 input query terms found dup hits:	[('SUGT1P1', 2), ('SNORD127', 2), ('SNORA62', 4), ('IFITM4P', 8), ('HLA-DRB6', 2), ('FUNDC2P2', 2), 
190 input query terms found no hit:	['NARFL', 'NFKBIL2', 'LOC150197', 'TMEM84', 'LOC162632', 'PPPDE1', 'PPPDE2', 'C1orf38', 'C1orf31', '
20 input query terms found dup hits:	[('PIP5K1P1', 2), ('SNORA40', 2), ('SNORD116-24', 2), ('SNORD116-25', 2), ('SNORD116-23', 2), ('SNOR
149 input query terms found no hit:	['FAM153C', 'C9orf167', 'CLK2P', 'CCDC76', 'CCDC75', 'CCDC72', 'HIST3H2BB', 'PRAC', 'LOC285780', 'LO
18 input query terms found dup hits:	[('SNORD58C', 2), ('UOX', 2), ('UBE2Q2P1', 3), ('PPP4R1L', 2), ('SNORD63', 3), ('SNORD116-22', 2), (
158 input query terms found no hit:	['

In [16]:
merged_data = selected_clinical_data.join(genetic_data.T).dropna()
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
sampleID,,,,,,,,,,,,,,,,,,,,,
TCGA-4C-A93U-01,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-2.090786,-0.086682,-0.748878,-1.018517,0.205567,-1.28139,0.395523,0.945273,-0.186695,4.024767
TCGA-BJ-A0YZ-01,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.503714,-0.086682,0.125322,1.568883,0.626367,2.38101,0.275823,2.140073,1.380405,1.257367
TCGA-BJ-A0Z0-01,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-1.602586,-0.086682,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.725173,-0.254095,0.530667
TCGA-BJ-A0Z2-01,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-1.516086,0.474118,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.894473,-0.380395,-2.502833
TCGA-BJ-A0Z3-01,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,0.971714,-0.086682,-0.413778,1.111583,1.158867,-1.28139,0.003723,0.924373,-0.143495,2.613367


In [17]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 572 samples.


In [18]:
is_trait_biased, merge_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0' with 59 occurrences. This represents 10.31% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.

Quartiles for 'Age':
  25%: 34.0
  50% (Median): 46.0
  75%: 58.0
Min: 15
Max: 89
The distribution of the feature 'Age' in this dataset is fine.

For the feature 'Gender', the least common label is '1' with 157 occurrences. This represents 27.45% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



False

In [19]:
merged_data.head()
if not is_trait_biased:
    merge_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

In [20]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

## 2.2. The GEO dataset

In GEO, there may be one or multiple cohorts for a trait. Each cohort is identified by an accession number. We iterate over all accession numbers in the corresponding subdirectory, preprocess the cohort data, and save them to csv files.

In [21]:
dataset = 'GEO'
trait_subdir = "Thyroid-Cancer"

trait_path = os.path.join(DATA_ROOT, dataset, trait_subdir)
os.listdir(trait_path)

['GSE191117',
 'GSE104005',
 'GSE151179',
 'GSE103996',
 'GSE138198',
 'GSE104006',
 'GSE151181',
 'GSE151180']

Repeat the below steps for all the accession numbers

In [175]:
cohort = accession_num = "GSE191117"
cohort_dir = os.path.join(trait_path, accession_num)
soft_file, matrix_file = get_relevant_filepaths(cohort_dir)
soft_file, matrix_file

('../../DATA/GEO/Thyroid-Cancer/GSE191117/GSE191117_family.soft.gz',
 '../../DATA/GEO/Thyroid-Cancer/GSE191117/GSE191117_series_matrix.txt.gz')

### Inital filtering and clinical data preprocessing

In [176]:
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']

background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
print(background_info)

!Series_title	"miRNA Expression Pattern in Papillary Thyroid Cancer"
!Series_summary	"The aim of this study was to conduct a comparison of tissue miRNA in thyroid cancer (papillary (PTC), follicular (FTC), and microcancer (PTMC)) and  healthy thyroid tissue (Control). The expression level of 798 miRNAs using NanoString technology was examined. ROC curve analysis, and logistic regression modeling were performed. Gene ontology (GO), canonical pathways analysis were used to explore the biological functions of the miRNA target genes."
!Series_summary	"The study revealed that 10 miRNAs were deregulated in samples of patients with PTC. Pathway analysis showed that miRNA target genes were mainly significantly enriched in endocrine resistance, EGFR tyrosine kinase inhibitor resistance, and pathways in cancer. ROC analysis demonstrated that miR-146-5p, miR-551b-3p, and miR-222-3p can be introduced as a diagnostic tool for PTC (AUC=0.770; 0.740; 0.720; respectively). Validation by qRT-PCR confir

In [177]:
clinical_data.head()

,!Sample_geo_accession,GSM5739405,GSM5739406,GSM5739407,GSM5739408,GSM5739409,GSM5739410,GSM5739411,GSM5739412,GSM5739413,...,GSM5739506,GSM5739507,GSM5739508,GSM5739509,GSM5739510,GSM5739511,GSM5739512,GSM5739513,GSM5739514,GSM5739515
0,!Sample_characteristics_ch1,disease state: PTC,disease state: PTC,disease state: Control,disease state: FTC,disease state: Control,disease state: PTC,disease state: Control,disease state: Control,disease state: PTMC,...,disease state: Control,disease state: Control,disease state: PTC,disease state: Control,disease state: PTC,disease state: PTC,disease state: PTC,disease state: Control,disease state: PTC,disease state: PTC
1,!Sample_characteristics_ch1,tnm staging system: pT1m,tnm staging system: pT2b,tissue: control (normal thyroid),tnm staging system: pT1m,tissue: control (normal thyroid),tnm staging system: pT1b,tissue: control (normal thyroid),tissue: control (normal thyroid),tnm staging system: pT1a,...,tissue: control (normal thyroid),tissue: control (normal thyroid),tnm staging system: pT1a,tissue: control (normal thyroid),tnm staging system: pT1a,tnm staging system: pT1a,tnm staging system: pT4,tissue: control (normal thyroid),tnm staging system: pT1b,tnm staging system: pT1b
2,!Sample_characteristics_ch1,tumor size: 7,tumor size: 14,NaN,tumor size: 5,NaN,tumor size: 9,NaN,NaN,tumor size: 9,...,NaN,NaN,tumor size: 10,NaN,tumor size: 9,tumor size: 9,tumor size: 25,NaN,tumor size: 15,tumor size: 15
3,!Sample_characteristics_ch1,multifocalization: 1-presence,multifocalization: 1-presence,NaN,multifocalization: 1-presence,NaN,multifocalization: 1-presence,NaN,NaN,multifocalization: 0-absence,...,NaN,NaN,multifocalization: 0-absence,NaN,multifocalization: 0-absence,lymph node metastasis: 1-presence,multifocalization: 1-presence,NaN,lymph node metastasis: 1-presence,multifocalization: 1-presence
4,!Sample_characteristics_ch1,angioinvasion: 1-presence,angioinvasion: 1-presence,NaN,angioinvasion: 0-absence,NaN,angioinvasion: 0-absence,NaN,NaN,angioinvasion: 0-absence,...,NaN,NaN,angioinvasion: 0-absence,NaN,angioinvasion: 0-absence,tissue: PTC,angioinvasion: 0-absence,NaN,tissue: PTC,angioinvasion: 1-presence


In [187]:
clinical_data_unique = get_unique_values_by_row(clinical_data)
clinical_data_unique

{0: ['disease state: PTC',
  'disease state: Control',
  'disease state: FTC',
  'disease state: PTMC'],
 1: ['tnm staging system: pT1m',
  'tnm staging system: pT2b',
  'tissue: control (normal thyroid)',
  'tnm staging system: pT1b',
  'tnm staging system: pT1a',
  'tnm staging system: no data',
  'tnm staging system: pT4',
  'tnm staging system: pT1',
  'tnm staging system: pT2',
  'tnm staging system: pT2a',
  'tnm staging system: pT3'],
 2: ['tumor size: 7',
  'tumor size: 14',
  nan,
  'tumor size: 5',
  'tumor size: 9',
  'tumor size: no data',
  'tumor size: 8',
  'tumor size: 10',
  'tumor size: 45',
  'tumor size: 3',
  'tumor size: 11',
  'tumor size: 20',
  'tumor size: 15',
  'tumor size: 6',
  'tumor size: 19',
  'tumor size: 1',
  'tumor size: 30',
  'tumor size: 13',
  'tumor size: 18',
  'tumor size: 25'],
 3: ['multifocalization: 1-presence',
  nan,
  'multifocalization: 0-absence',
  'multifocalization: no data',
  'lymph node metastasis: 1-presence'],
 4: ['angioinv

Analyze the metadata to determine data relevance and find ways to extract the clinical data.
Reference prompt:

In [188]:
f'''As a biomedical research team, we are selecting datasets to study the association between the human trait \'{TRAIT}\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:
1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)
2. For each of the traits \'{TRAIT}\', 'age', and 'gender', please address these points:
   (1) Is there human data available for this trait?
   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is an integer. The trait information might be explicitly recorded, or can be inferred from the field with some biomedical knowledge or understanding about the data collection process.
   (3) Choose an appropriate data type (either 'continuous' or 'binary') for each trait. Write a Python function to convert any given value of the trait to this data type. The function should handle inference about the trait value and convert unknown values to None.
   Name the functions 'convert_trait', 'convert_age', and 'convert_gender', respectively.
   NOTE: Function "convert_trait" should return 1 or 0.
   
Background information about the dataset:
{background_info}

Sample characteristics dictionary (from "!Sample_characteristics_ch1", converted to a Python dictionary that stores the unique values for each field):
{clinical_data_unique}
'''

'As a biomedical research team, we are selecting datasets to study the association between the human trait \'Thyroid Cancer\' and genetic factors, optionally considering the influence of age and gender. After searching the GEO database and parsing the matrix file of a series, we obtained background information and sample characteristics data. We will provide textual information about the dataset background, and a Python dictionary storing a list of unique values for each field of the sample characteristics data. Please carefully review the provided information and answer the following questions about this dataset:\n1. Does this dataset contain gene expression data? (Note: Pure miRNA data is not suitable.)\n2. For each of the traits \'Thyroid Cancer\', \'age\', and \'gender\', please address these points:\n   (1) Is there human data available for this trait?\n   (2) If so, identify the key in the sample characteristics dictionary where unique values of this trait is recorded. The key is

Understand and verify the answer from GPT, to assign values to the below variables. Assign None to the 'row_id' variables if relevant data row was not found.
Later we need to let GPT format its answer to automatically do these. But given the complexity of this step, let's grow some insight from the free-text answers for now.

In [180]:
age_row = gender_row = None
convert_age = convert_gender = None

In [183]:
is_gene_availabe = True
trait_row = 0
age_row = None
gender_row = None

trait_type = 'binary'

In [184]:
is_available = is_gene_availabe and (trait_row is not None)
if not is_available:
    save_cohort_info(cohort, JSON_PATH, is_available)
    print("This cohort is not usable. Please skip the following steps and jump to the next accession number.")

In [189]:
def convert_trait(value):
    if 'Control' in value:
        return 0  # presence of thyroid cancer
    else:
        return 1  # absence of thyroid cancer

def convert_age(value):
    if value.startswith('age: '):
        return int(value.split(': ')[1])
    else:
        return None

def convert_gender(value):
    if value == 'Sex: M':
        return 0
    elif value == 'Sex: F':
        return 1
    else:
        return None

In [190]:
selected_clinical_data = geo_select_clinical_features(clinical_data, TRAIT, trait_row, convert_trait, age_row=age_row,
                                                      convert_age=convert_age, gender_row=gender_row,
                                                      convert_gender=convert_gender)
selected_clinical_data.head()

,GSM5739405,GSM5739406,GSM5739407,GSM5739408,GSM5739409,GSM5739410,GSM5739411,GSM5739412,GSM5739413,GSM5739414,...,GSM5739506,GSM5739507,GSM5739508,GSM5739509,GSM5739510,GSM5739511,GSM5739512,GSM5739513,GSM5739514,GSM5739515
Thyroid Cancer,1,1,0,1,0,1,0,0,1,0,...,0,0,1,0,1,1,1,0,1,1


### Genetic data preprocessing and final filtering

In [191]:
genetic_data = get_genetic_data(matrix_file)
genetic_data.head()

,GSM5739405,GSM5739406,GSM5739407,GSM5739408,GSM5739409,GSM5739410,GSM5739411,GSM5739412,GSM5739413,GSM5739414,...,GSM5739506,GSM5739507,GSM5739508,GSM5739509,GSM5739510,GSM5739511,GSM5739512,GSM5739513,GSM5739514,GSM5739515
ID,,,,,,,,,,,,,,,,,,,,,
ACTB,20.000,481.473,20.000,1073.345,1990.023,846.540,110.106,20.000,97.238,20.000,...,3988.621,101.708,1101.062,5062.791,1318.050,142.817,97.150,46.242,224.837,800.849
ath-miR159a,20.000,24.440,827.367,29.172,22.061,33.254,36.523,249.451,106.725,338.793,...,20.000,56.207,20.000,20.000,20.000,132.616,20.000,20.000,20.000,20.000
B2M,618.571,2777.633,20.000,1589.395,6467.738,1408.050,1684.896,583.942,939.181,20.000,...,13520.488,248.917,5882.497,13687.668,3828.977,469.255,4242.232,249.706,2144.604,4225.565
cel-miR-248,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000,...,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000,20.000
cel-miR-254,1858696.125,107737.461,3510915.000,87826.172,41849.531,62179.359,37410.375,493986.875,176805.484,964194.438,...,112463.328,558705.250,199338.953,200832.938,141009.797,982222.688,362681.719,140227.531,41983.695,34797.746


In [192]:
gene_row_ids = genetic_data.index[:20].tolist()
gene_row_ids

['ACTB',
 'ath-miR159a',
 'B2M',
 'cel-miR-248',
 'cel-miR-254',
 'GAPDH',
 'hsa-let-7a-5p',
 'hsa-let-7b-5p',
 'hsa-let-7c-5p',
 'hsa-let-7d-5p',
 'hsa-let-7e-5p',
 'hsa-let-7f-5p',
 'hsa-let-7g-5p',
 'hsa-let-7i-5p',
 'hsa-miR-1-3p',
 'hsa-miR-1-5p',
 'hsa-miR-100-5p',
 'hsa-miR-101-3p',
 'hsa-miR-103a-3p',
 'hsa-miR-105-5p']

Check if the gene dataset requires mapping to get the gene symbols corresponding to each data row.

Reference prompt:

In [193]:
f'''
Below are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:
requires_gene_mapping = (True or False)

Row headers:
{gene_row_ids}
'''

"\nBelow are the row headers of a gene expression dataset in GEO. Based on your biomedical knowledge, are they human gene symbols, or are they some other identifiers that need to be mapped to gene symbols? Your answer should be concluded by starting a new line and strictly following this format:\nrequires_gene_mapping = (True or False)\n\nRow headers:\n['ACTB', 'ath-miR159a', 'B2M', 'cel-miR-248', 'cel-miR-254', 'GAPDH', 'hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hsa-let-7d-5p', 'hsa-let-7e-5p', 'hsa-let-7f-5p', 'hsa-let-7g-5p', 'hsa-let-7i-5p', 'hsa-miR-1-3p', 'hsa-miR-1-5p', 'hsa-miR-100-5p', 'hsa-miR-101-3p', 'hsa-miR-103a-3p', 'hsa-miR-105-5p']\n"

If not required, jump directly to the gene normalization step

In [194]:
requires_gene_mapping = True

In [195]:
if requires_gene_mapping:
    gene_annotation = get_gene_annotation(soft_file)
    gene_annotation_summary = preview_df(gene_annotation)
    print(gene_annotation_summary)

{'ID': ['hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hsa-let-7d-5p', 'hsa-let-7e-5p'], 'miRNA_ID_LIST': ['hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hsa-let-7d-5p', 'hsa-let-7e-5p'], 'Accession': ['MIMAT0000062', 'MIMAT0000063', 'MIMAT0000064', 'MIMAT0000065', 'MIMAT0000066'], 'Code Class': ['Endogenous', 'Endogenous', 'Endogenous', 'Endogenous', 'Endogenous'], 'SPOT_ID': [nan, nan, nan, nan, nan]}


Observe the first few cells in the ID column of the gene annotation dataframe, to find the names of columns that store the gene probe IDs and gene symbols respectively.
Reference prompt:

In [136]:
# if requires_gene_mapping:
f'''
As a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:
probe_name_key = key_name1
gene_name_key = key_name2

Gene annotation dictionary:
{gene_annotation_summary}
'''

"\nAs a biomedical research team, we extracted the gene annotation data from a series in the GEO database, and saved it to a Python dictionary. Please read the dictionary, and decide which key stores the ID of the probe, and which key stores the gene symbols. Please strict follow this format in your answer:\nprobe_name_key = key_name1\ngene_name_key = key_name2\n\nGene annotation dictionary:\n{'ID': ['1', '2', '3', '4', '5'], 'COL': ['192', '192', '192', '192', '192'], 'ROW': [328.0, 326.0, 324.0, 322.0, 320.0], 'SPOT_ID': ['miRNABrightCorner30', 'Blank', 'Blank', 'Blank', 'Blank'], 'CONTROL_TYPE': ['pos', 'pos', 'pos', 'pos', 'pos'], 'miRNA_ID': [nan, nan, nan, nan, nan], 'GENE_SYMBOL': [nan, nan, nan, nan, nan], 'GENE_NAME': [nan, nan, nan, nan, nan], 'ACCESSION_STRING': [nan, nan, nan, nan, nan], 'CHROMOSOMAL_LOCATION': [nan, nan, nan, nan, nan]}\n"

In [138]:
if requires_gene_mapping:
    probe_id_key = 'ID'
    gene_symb_key = 'GENE_NAME'
    gene_mapping = get_gene_mapping(gene_annotation, probe_id_key, gene_symb_key)
    genetic_data = apply_gene_mapping(genetic_data, gene_mapping)

In [196]:
if NORMALIZE_GENE:
    genetic_data = normalize_gene_symbols_in_index(genetic_data)

822 input query terms found no hit:	['ath-miR159a', 'cel-miR-248', 'cel-miR-254', 'hsa-let-7a-5p', 'hsa-let-7b-5p', 'hsa-let-7c-5p', 'hs


In [197]:
merged_data = geo_merge_clinical_genetic_data(selected_clinical_data, genetic_data)
# The preprocessing runs through, which means is_available should be True
is_available = True

In [198]:
print(f"The merged dataset contains {len(merged_data)} samples.")

The merged dataset contains 111 samples.


In [199]:
is_trait_biased, merged_data = judge_and_remove_biased_features(merged_data, TRAIT, trait_type=trait_type)
is_trait_biased

For the feature 'Thyroid Cancer', the least common label is '0.0' with 50 occurrences. This represents 45.05% of the dataset.
The distribution of the feature 'Thyroid Cancer' in this dataset is fine.



False

In [200]:
save_cohort_info(cohort, JSON_PATH, is_available, is_trait_biased, merged_data)

In [201]:
merged_data.head()
if not is_trait_biased:
    merged_data.to_csv(os.path.join(OUTPUT_DIR, cohort+'.csv'), index=False)

### 3. Do regression & Cross Validation

In [4]:
# Check the information of usable cohorts
best_cohort, ranked_df = filter_and_rank_cohorts(JSON_PATH)
ranked_df

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,572,
1,GSE191117,True,True,False,False,False,111,
3,GSE151179,True,True,False,False,False,52,
7,GSE151181,True,True,False,False,False,51,
8,GSE151180,True,True,False,False,False,46,
2,GSE104005,True,True,False,True,True,34,
4,GSE103996,True,True,False,True,True,34,
6,GSE104006,True,True,False,True,True,34,


In [5]:
# If both age and gender have available cohorts, select 'age' as the condition.
condition = 'Age'
filter_column = 'has_' + condition.lower()

condition_best_cohort, condition_ranked_df = filter_and_rank_cohorts(JSON_PATH, filter_column)
condition_best_cohort

'Xena'

In [6]:
condition_ranked_df.head()

,cohort_id,is_usable,is_available,is_biased,has_age,has_gender,sample_size,note
0,Xena,True,True,False,True,True,572,
2,GSE104005,True,True,False,True,True,34,
4,GSE103996,True,True,False,True,True,34,
6,GSE104006,True,True,False,True,True,34,


In [7]:
merged_data = pd.read_csv(os.path.join(OUTPUT_DIR, condition_best_cohort + '.csv'))
merged_data.head()

,Thyroid Cancer,Age,Gender,ARHGEF10L,HIF3A,RNF17,RNF10,RNF11,RNF13,GTF2IP1,...,SLC7A10,PLA2G2C,TULP2,NPY5R,GNGT2,GNGT1,TULP3,BCL6B,GSTK1,SELP
0,1,74,0,-0.767592,-1.742926,-0.531035,-0.067972,0.265922,-0.13531,0.440006,...,-2.090786,-0.086682,-0.748878,-1.018517,0.205567,-1.28139,0.395523,0.945273,-0.186695,4.024767
1,1,65,1,1.766308,0.273374,-0.531035,0.790028,1.209322,0.01229,0.704606,...,0.503714,-0.086682,0.125322,1.568883,0.626367,2.38101,0.275823,2.140073,1.380405,1.257367
2,1,55,1,0.986808,-2.165726,-0.050935,0.051428,0.979522,0.48229,0.090106,...,-1.602586,-0.086682,-0.748878,-0.747417,-0.931133,-1.28139,-0.222077,0.725173,-0.254095,0.530667
3,1,57,1,0.767208,-1.623626,-0.531035,0.182228,0.541622,0.65889,0.647206,...,-1.516086,0.474118,-0.748878,-1.587117,-1.292433,-1.28139,-0.602177,0.894473,-0.380395,-2.502833
4,1,33,0,-0.287292,-1.766626,-0.531035,0.059628,0.920522,0.21849,0.336106,...,0.971714,-0.086682,-0.413778,1.111583,1.158867,-1.28139,0.003723,0.924373,-0.143495,2.613367


In [8]:
# Remove the other condition to prevent interference.
merged_data = merged_data.drop(columns=['Gender'], errors='ignore').astype('float')

X = merged_data.drop(columns=[TRAIT, condition]).values
Y = merged_data[TRAIT].values
Z = merged_data[condition].values

Select the appropriate regression model depending on whether the dataset shows batch effect.

In [9]:
has_batch_effect = detect_batch_effect(X)
has_batch_effect

True

In [10]:
# Select appropriate models based on whether the dataset has batch effect.
# We experiment on two models for each branch. We will decide which one to choose later.

if has_batch_effect:
    model_constructor1 = VariableSelection
    model_params1 = {'modified': True, 'lamda': 3e-4}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}
else:
    model_constructor1 = Lasso
    model_params1 = {'alpha': 1.0, 'random_state': 42}
    model_constructor2 = VariableSelection
    model_params2 = {'modified': False}

In [11]:
trait_type = 'binary'  # Remember to set this properly, either 'binary' or 'continuous'
cv_mean1, cv_std1 = cross_validation(X, Y, Z, model_constructor1, model_params1, target_type=trait_type)

alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
alpha for Lasso: 0.0003
The cross-validation accuracy is 51.05% ± 2.96%


In [12]:
cv_mean2, cv_std2 = cross_validation(X, Y, Z, model_constructor2, model_params2, target_type=trait_type)

/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)
/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


The cross-validation accuracy is 82.63% ± 3.06%


In [13]:
normalized_X, _ = normalize_data(X)
normalized_Z, _ = normalize_data(Z)

# Train regression model on the whole dataset to identify significant genes
model1 = ResidualizationRegressor(model_constructor1, model_params1)
model1.fit(normalized_X, Y, normalized_Z)

model2 = ResidualizationRegressor(model_constructor2, model_params2)
model2.fit(normalized_X, Y, normalized_Z)

alpha for Lasso: 0.0003


/Users/ollie/anaconda3/lib/python3.11/site-packages/sparse_lmm/helpingMethods.py:49: RuntimeWarning: invalid value encountered in divide
  ts = beta / np.sqrt(var * sigma)


### 4. Discussion and report

In [14]:
feature_cols = merged_data.columns.tolist()
feature_cols.remove(TRAIT)

threshold = 0.05
interpret_result(model1, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=1)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0112
Found 94 genes with non-zero coefficients associated with the trait 'Thyroid Cancer' conditional on the factor 'Age'. These genes are identified as significant based on the regression model.
  Variable  Coefficient  Absolute Coefficient
    SPATA1    -1.591095              1.591095
      ODF4     1.276682              1.276682
    BARHL2    -1.051818              1.051818
   SULT1B1    -1.017430              1.017430
   FAM153A     0.949215              0.949215
   DCAF4L2     0.938226              0.938226
      HBG2     0.869382              0.869382
  FAM41AY1    -0.851217              0.851217
    CT45A1     0.796943              0.796943
  DEFB108B    -0.791760              0.791760
 RPL13AP17     0.768681              0.768681
     OR4C3     0.736638              0.736638
     UBL4B    -0.735610              0.735610
    RNF151     0.731442              0.731442
    UGT1A7     0.646855              

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:469: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df['Absolute Coefficient'] = gene_regression_df['Coefficient'].abs()


In [15]:
interpret_result(model2, feature_cols, TRAIT, condition, threshold=threshold, save_output=True,
                 output_dir=OUTPUT_DIR, model_id=2)

Effect of the condition on the target variable:
Variable: Age
Coefficient: 0.0112
Found 79 significant genes associated with the trait 'Thyroid Cancer' conditional on the factor 'Age', with corrected p-value < 0.05:
   Variable  Coefficient      p_value  corrected_p_value
     GABRB2     0.200481 1.324083e-27       2.295827e-23
       LRP4     0.151621 1.103395e-14       9.565882e-11
    SHROOM4     0.148440 1.732095e-12       1.001093e-08
      HMGA2     0.121740 1.847765e-11       8.009598e-08
       EPS8     0.148901 1.041999e-10       3.613444e-07
    KLHDC8A     0.102289 2.549553e-10       7.367785e-07
       LIPH     0.125905 3.460969e-10       8.572819e-07
      MPZL2     0.123121 4.852839e-10       1.051792e-06
      GABRD     0.084304 3.759076e-09       7.242070e-06
       GJB3     0.102627 6.973397e-09       1.209117e-05
      KCNQ3     0.133119 1.572177e-08       2.478179e-05
        FN1     0.138885 4.020338e-08       5.809053e-05
    ZCCHC12     0.089829 5.219035e-08      

/Users/ollie/Documents/JupyterProjects/AI4Science_gold_standard/stage1/Jinglin/utils.py:478: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  gene_regression_df.loc[:, 'corrected_p_value'] = corrected_p_values
